### Read .dat file generated from box and confirm impure probabilities

In [12]:
import numpy as np
import pandas as pd
from pathlib import Path
from banditpy.core import Bandit2Arm
from banditpy.io import dat2ArmIO, csv2ArmIO


basedir = Path("D:/Data/mab/BGdataset")
animals = [basedir / "BGM0", basedir / "BGM1", basedir / "BGM2"]
# animals = [basedir / "BGM1"]

tasks: list[Bandit2Arm] = [csv2ArmIO(folder / "raw_data") for folder in animals]

# basedir = Path("D:/Data/mab/AAdataset/aguero/raw_data")

# task = dat2ArmIO(basedir)

[WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-17-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-18-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-21-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-22-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-23-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-24-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-25-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-26-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-27-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-28-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/raw_data/BGM0-2025-09-29-16-32-41.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM0/ra

C:\Users\asheshlab\Documents\Codes\BanditPy\banditpy\io\csvio.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat(dfs, ignore_index=True)


nfiles=40
[1 2]
655368
[WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-17-17-13-09.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-18-17-13-09.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-19-17-13-09.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-20-17-13-09.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-21-17-13-09.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-22-17-13-09.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-23-17-13-09.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-24-17-13-09.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-25-17-13-09.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-26-17-13-09.trial.csv'), WindowsPath('D:/Data/mab/BGdataset/BGM2/raw_data/BGM2-2025-09-27-17-13-09.trial.csv')]
nfiles=11
[2 1]
198

In [3]:
tasks[0].to_df()

,choices,rewards,probs_1,probs_2,session_ids,starts,stops,datetime
0,1,0,0.2,0.4,1,None,None,None
1,1,0,0.2,0.4,1,None,None,None
2,1,0,0.2,0.4,1,None,None,None
3,2,1,0.2,0.4,1,None,None,None
4,1,0,0.2,0.4,1,None,None,None
...,...,...,...,...,...,...,...,...
79215,2,0,0.7,0.8,605,None,None,None
79216,2,1,0.7,0.8,605,None,None,None
79217,2,1,0.7,0.8,605,None,None,None
79218,2,1,0.7,0.8,605,None,None,None


In [47]:
from neuropy import plotting
from datetime import datetime, timezone

fig = plotting.Fig(5, 1)
bin_edges = np.linspace(0, 24, 49)  # 0,0.5,...,24
bin_centers = bin_edges[:-1] + 0.25

for i, task in enumerate(tasks):

    dttime = task.to_df()["datetime"].to_numpy()
    time = np.array([datetime.fromtimestamp(ts, tz=timezone.utc) for ts in dttime])
    dt_series = pd.to_datetime(time, errors="coerce")
    hours = (
        dt_series.hour.to_numpy()
        + dt_series.minute.to_numpy() / 60.0
        + dt_series.second.to_numpy() / 3600.0
    )
    counts, _ = np.histogram(hours, bins=bin_edges)
    ax = fig.subplot(fig.gs[i])
    ax.bar(
        bin_centers, counts, width=0.48, align="center", edgecolor="k", linewidth=0.3
    )
    # ax.plot(time, ".")

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime


def read_datfile(folder):
    dfs = [pd.read_csv(fp, sep=",", header=None) for fp in sorted(folder.glob("*.dat"))]
    data = pd.concat(dfs, ignore_index=True)
    return data


df = read_datfile(animals[0])

arr = df[5].to_numpy()
dt = [datetime.fromtimestamp(ts) for ts in arr]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(tasks[0].datetime)

In [ ]:
from datetime import datetime, timezone
import pandas as pd

val = 1756913711000
dt = datetime.fromtimestamp(val / 1000, tz=timezone.utc)

pd.to_datetime(val, unit="ms", origin="unix", utc=True)

In [14]:
import matplotlib.pyplot as plt
from banditpy.plots import plot_trial_by_trial_2Arm
from neuropy import plotting
from scipy import stats

fig = plotting.Fig(5, 4)
animal_names = ["BGM0", "BGM1", "BGM2"]

for i, task in enumerate(tasks):

    probs = task.probs[task.is_session_start, :]
    probs_corr = stats.pearsonr(probs[:, 0].astype(float), probs[:, 1].astype(float))
    print(probs_corr[0])

    p1_bins = np.linspace(0, 0.9, 10) + 0.05
    p2_bins = np.linspace(0, 0.9, 10) + 0.05
    H, xedges, yedges, _ = stats.binned_statistic_2d(
        probs[:, 0].astype(float),
        probs[:, 1].astype(float),
        values=probs[:, 0],
        statistic="count",
        bins=[p1_bins, p2_bins],
    )

    choices = task.choices.copy()
    # choices[choices == 1] = 2
    # choices[task.choices == 2] = 1
    # task.choices = choices

    perf = task.filter_by_trials(
        min_trials=100, clip_max=100
    ).get_optimal_choice_probability()

    ax = fig.subplot(fig.gs[i, 0])
    ax.plot(perf, color="k")
    # task.plot_trial_by_trial(ax=ax)
    # ax.set_ylim(0.1, 0.7)
    ax.set_xlabel("Trial_id")
    ax.set_ylabel("Choice (High)")
    ax.set_title(f"{animal_names[i]}'s performance")

    ax2 = fig.subplot(fig.gs[i, 1])
    cplot = ax2.pcolormesh(
        xedges,
        yedges,
        H.T,
        cmap="hot",
        vmin=0,
        # vmax=14,
        shading="auto",
    )
    cb = plt.colorbar(cplot, ax=ax2, shrink=0.5)
    # ax2.imshow(prob_mat)
    ax2.set_xlabel("Prob1")
    ax2.set_ylabel("Prob2")
    ax2.set_title(f"{animal_names[i]}'s probability combinations")
    # ax2.set_xticks([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
    # ax2.set_yticks([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
    cb.set_label("Counts")

# plt.plot(perf)
# plot_trial_by_trial_2Arm(task)

-0.16159979712104394
-0.7885212454813333
-0.30187734478740563


In [ ]:
probs[:, 0]

array([0.2, 0.2, 0.7, 0.6, 0.4, 0.8, 0.6, 0.6, 0.6, 0.3, 0.4, 0.3, 0.6,
       0.4, 0.6, 0.3, 0.7, 0.6, 0.2, 0.6, 0.8, 0.6, 0.8, 0.4, 0.4, 0.3,
       0.7, 0.4, 0.3, 0.6, 0.3, 0.6, 0.6, 0.7, 0.3, 0.6, 0.8, 0.3, 0.2,
       0.6, 0.8, 0.2, 0.2, 0.8, 0.7, 0.7, 0.4, 0.3, 0.7, 0.6, 0.7, 0.8,
       0.2, 0.6, 0.7, 0.4, 0.6, 0.6, 0.2, 0.7, 0.7, 0.4, 0.6, 0.7, 0.6,
       0.3, 0.8, 0.7, 0.2, 0.3, 0.8, 0.6, 0.2, 0.4, 0.3, 0.7, 0.6, 0.6,
       0.8, 0.2, 0.4, 0.6, 0.6, 0.6, 0.2, 0.2, 0.7, 0.2, 0.7, 0.2, 0.3,
       0.2, 0.3, 0.6, 0.6, 0.8, 0.4, 0.6, 0.3, 0.6, 0.4, 0.3, 0.2, 0.7,
       0.4, 0.2, 0.6, 0.6, 0.4, 0.7, 0.3, 0.4, 0.3, 0.3, 0.4, 0.6, 0.7,
       0.8, 0.2, 0.6, 0.2, 0.2, 0.4, 0.4, 0.3, 0.6, 0.6, 0.2, 0.3, 0.7,
       0.2, 0.7, 0.7, 0.3, 0.7, 0.4, 0.2, 0.7, 0.2, 0.8, 0.4, 0.3, 0.2,
       0.2, 0.3, 0.3, 0.4, 0.6, 0.3, 0.6, 0.2, 0.7, 0.4, 0.4, 0.2, 0.2,
       0.8, 0.4, 0.4, 0.7, 0.2, 0.3, 0.2, 0.3, 0.7, 0.8, 0.6, 0.8, 0.3,
       0.7, 0.2, 0.8, 0.3, 0.2, 0.2, 0.8, 0.3, 0.2, 0.3, 0.3, 0.

In [ ]:
tasks[1].n_sessions

In [ ]:
tasks[1].ntrials_session

In [ ]:
tasks[1].to_df()

In [ ]:
np.diff(tasks[2].datetime[tasks[2].is_session_start])

In [ ]:
tasks[2].ntrials_session

### Debugging

In [44]:
import numpy as np
import pandas as pd
from pathlib import Path
from banditpy.core import Bandit2Arm
from banditpy.io import dat2ArmIO, csv2ArmIO


basedir = Path("D:/Data/mab/BGdataset")
# animals = [basedir / "BGM0", basedir / "BGM1", basedir / "BGM2"]
animals = [basedir / "BGF0"]

# tasks: list[Bandit2Arm] = [csv2ArmIO(folder / "raw_data") for folder in animals]

folder = Path("D:/Data/mab/BGdataset/BGF0/raw_data")
files = sorted(folder.glob("*.csv"))
print(len(files))
dfs = [pd.read_csv(fp, sep=",") for fp in files]
print(f"nfiles={len(dfs)}")
data = pd.concat(dfs, ignore_index=True)

data = data[
    (data["eventCode"].astype(str).str.contains("200"))
    & (data["chosenPort"].isin([1, 2]))
]

# blockId = data["blockId"].to_numpy()
# block_starts = np.where(np.diff(blockId, prepend=-1) != 0, 1, 0)
# sessionId = np.cumsum(block_starts)

# print(data["chosenPort"].unique())
print(data.size)

# b2a = Bandit2Arm(
#     probs=data[["port1Prob", "port2Prob"]].to_numpy(),
#     choices=data["chosenPort"].to_numpy(),
#     rewards=data["rewarded"].to_numpy(),
#     block_ids=blockId,
#     session_ids=sessionId,
#     # window_ids=data["window_id"].to_numpy(),
#     # datetime=data["datetime"].to_numpy(),
# )

34
nfiles=34
37608
